In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [4]:
#Loading the data
data=pd.read_csv('F:/IMP.DATA/Documents/FlippNation/r/Data (1)/Data/bank-full.csv',sep=';')
data.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
#Converting the format of Output Variable from Strings to Numeric
# 
data.y.replace(('yes','no'),(1,0),inplace=True)
data.default.replace(('yes','no'),(1,0),inplace=True)
data.housing.replace(('yes','no'),(1,0),inplace=True)
data.loan.replace(('yes','no'),(1,0),inplace=True)

data.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0


In [10]:
#Concatenating the dummy variables for the feature job with several levels and the original dataset 
data12=pd.concat([pd.get_dummies(data.job),data.iloc[:,0:17]],axis=1)
#Concatenating the dummy variables for the feature marital with three levels and the modified dataset (data12)
data13=pd.concat([pd.get_dummies(data12.marital),data12.iloc[:,0:29]],axis=1)
#Concatenating the dummy variables for the feature education with four levels and the modified dataset (data13)
data14=pd.concat([pd.get_dummies(data13.education),data13.iloc[:,0:32]],axis=1)
#Concatenating the dummy variables for the feature contact with three levels and the modified dataset (data14)
data15=pd.concat([pd.get_dummies(data14.contact),data14.iloc[:,0:36]],axis=1)
#Concatenating the dummy variables for the feature month with twelve levels and the modified dataset (data15)
data16=pd.concat([pd.get_dummies(data15.month),data15.iloc[:,0:39]],axis=1)
#Concatenating the dummy variables for the feature poutcome with four levels and the modified dataset (data15)
data17=pd.concat([pd.get_dummies(data16.poutcome),data16.iloc[:,0:51]],axis=1)
data17.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 55 columns):
failure          45211 non-null uint8
other            45211 non-null uint8
success          45211 non-null uint8
unknown          45211 non-null uint8
apr              45211 non-null uint8
aug              45211 non-null uint8
dec              45211 non-null uint8
feb              45211 non-null uint8
jan              45211 non-null uint8
jul              45211 non-null uint8
jun              45211 non-null uint8
mar              45211 non-null uint8
may              45211 non-null uint8
nov              45211 non-null uint8
oct              45211 non-null uint8
sep              45211 non-null uint8
cellular         45211 non-null uint8
telephone        45211 non-null uint8
unknown          45211 non-null uint8
primary          45211 non-null uint8
secondary        45211 non-null uint8
tertiary         45211 non-null uint8
unknown          45211 non-null uint8
divorced         

In [145]:
#Final Features (Numerical)
#Removing the duplicacy by dropping the original factor variables 
#converting the data to an array using .values to make the features compatible with the Logistic Regression
#model
features=data17.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,42,43,44,45,47,49,50,51,52]]
features.values

array([[  0,   0,   0, ...,   1,  -1,   0],
       [  0,   0,   0, ...,   1,  -1,   0],
       [  0,   0,   0, ...,   1,  -1,   0],
       ...,
       [  0,   0,   1, ...,   5, 184,   3],
       [  0,   0,   0, ...,   4,  -1,   0],
       [  0,   1,   0, ...,   2, 188,  11]], dtype=int64)

In [144]:
#Create features and labels
#For classification, features must be n(d) array and label must be 1d array

#Label
label=data17.y
label

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
45181    0
45182    1
45183    0
45184    0
45185    1
45186    0
45187    1
45188    1
45189    0
45190    1
45191    1
45192    1
45193    1
45194    0
45195    1
45196    1
45197    1
45198    0
45199    0
45200    1
45201    1
45202    1
45203    1
45204    1
45205    1
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64

In [149]:

#Making a for loop where the data is split in train/test to cross validate the logistic model that we create 
#As well as Generalization of the Model to get a test score higher than the train score

warnings.filterwarnings("ignore")
for i in range (0,10):
    X_train,X_test,Y_train,Y_test=train_test_split(features,
                                                  label,
                                                  test_size=0.2,
                                                  random_state=i)
    model = LogisticRegression()
    model.fit(X_train,Y_train)
    
    train_score=model.score(X_train,Y_train)
    test_score=model.score(X_test,Y_test)
    
    if test_score>train_score:
        print("Test Score:{} Train Score:{} Random State:{}".format(test_score,train_score,i))

Test Score:0.9027977441114674 Train Score:0.901598097765981 Random State:2
Test Score:0.9032400751962845 Train Score:0.9012386640123866 Random State:3
Test Score:0.9030189096538759 Train Score:0.9016257465162575 Random State:4
Test Score:0.9043459029083268 Train Score:0.9011833665118336 Random State:5
Test Score:0.9098750414685391 Train Score:0.9001880115018801 Random State:7


In [153]:
#The random state 7 is giving the highest generalization
#Train Test Split for the model
X_train, X_test, y_train, y_test = train_test_split(features,
                                                   label,
                                                   test_size=0.2,
                                                   random_state=7)

In [155]:
#Implement Linear Regression
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
data.y.value_counts()

0    39922
1     5289
Name: y, dtype: int64

In [159]:
#The confusion Matrix for the Classification Model that we built
# So the TPR/Recall/Sensitivity (0's) accurately predicted are 38945 out of 39922(97.55%)(Did not subscribe to the term deposit)
# the TNR/Specificity (1's) accurately predicted are 1841 out of 5289(34.80%)(Subscribed to the term deposit)
# Sensitivity  of 0 is 0.98 and Specificity of 1 is 0.35 
# The Precision of the model,total number of correctly predicted (0's) are 38945 out of 42393(92.0 %)
# The Precision of the model,total number of correctly predicted (1's) are 1841 out of 5289(65.33 %)

cm = confusion_matrix(label,model.predict(features))
cm

array([[38945,   977],
       [ 3448,  1841]], dtype=int64)

In [161]:
#From the classification report below 
#The accuracy of the model is 90 percent 

#print(classification_report())
print(classification_report(label,model.predict(features)))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95     39922
           1       0.65      0.35      0.45      5289

    accuracy                           0.90     45211
   macro avg       0.79      0.66      0.70     45211
weighted avg       0.89      0.90      0.89     45211

